In [1]:
import sys, os 
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
from tactic_gen.train_codellama import get_trainer
from tactic_gen.train_codellama import get_tokenizer
from transformers import CodeLlamaTokenizer

/home/ubuntu/coq-modeling/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
conf = {
    "data_path": "/home/ubuntu/coq-modeling/data/data-basic",
    "model_name": "codellama/CodeLlama-7b-hf",
    "output_dir": "/home/ubuntu/coq-modeling/models/test",

    # Training Args
    "max_seq_len": 512,
    "max_input_len": 448,
    "per_device_train_batch_size": 4,
    "learning_rate": 1.41e-5,
    "num_train_epochs": 1,
    "peft_lora_r": 64,
    "peft_lora_alpha": 16,

    # Evaluation Args
    "eval_steps": 500,
    "eval_accumulation_steps": 1,
    "per_device_eval_batch_size": 2,
    "num_eval_examples": 2000, # Evaluation would take ~2 hours each time w/o limiting this

    # Logging Args
    "logging_steps": 10,
    "save_steps": 100,
    "save_total_limit": 5,
}

In [ ]:
model_name = conf["model_name"] 
seq_len = conf["max_seq_len"] 
tokenizer: CodeLlamaTokenizer = CodeLlamaTokenizer.from_pretrained(model_name)
tokenizer.add_eos_token = True

pad_token = "_<PRE>" 
encoded_ids = tokenizer.encode(pad_token)

In [3]:
tokenizer = get_tokenizer(conf)

AssertionError: 

In [3]:
trainer = get_trainer(conf)



Building Training Config...
gradient_accumulation_steps not found in configuration. Defaulting to 2
max_steps not found in configuration. Defaulting to -1


Retrieving Model...


Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.34s/it]




Retrieving Tokenizer...


AssertionError: 

In [4]:
trainer.train_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 164135
})

In [14]:
toks = trainer.tokenizer.tokenize("hello-moto there bum")
print(toks)
trainer.tokenizer.convert_tokens_to_string(toks[3:])

['▁hello', '-', 'm', 'oto', '▁there', '▁b', 'um']


'oto there bum'

In [30]:
trainer.tokenizer.all_special_ids

[1, 2, 0, 2, 32007, 32009, 32008, 32010]

In [12]:
trainer.tokenizer.eos_token in trainer.train_dataset[1]["input_ids"]

False